In [3]:
import pymysql
import json
from typing import List

def get_reference_documents(cursor, source_id: str) -> List[str]:
    """reference_documents 테이블에서 해당 source_document_id의 모든 reference_document_id 조회"""
    query = """
        SELECT reference_document_id 
        FROM reference_documents 
        WHERE source_document_id = %s
    """
    cursor.execute(query, (source_id,))
    results = cursor.fetchall()
    return [str(row['reference_document_id']) for row in results]

def update_documents_references():
    """documents 테이블의 references를 reference_documents 테이블 데이터로 업데이트"""
    
    # DB 연결
    connection = pymysql.connect(
        host='localhost',
        user='root',  # 사용자명 수정
        password='1234',  # 비밀번호 수정
        database='any_approval',
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
    
    try:
        with connection.cursor() as cursor:
            # documents 테이블에서 모든 레코드 조회
            cursor.execute("SELECT id, source_id FROM documents")
            documents = cursor.fetchall()
            
            total_count = len(documents)
            update_count = 0
            no_reference_count = 0
            
            print(f"총 {total_count}개 문서 처리 시작...\n")
            
            for idx, doc in enumerate(documents, 1):
                doc_id = doc['id']
                source_id = doc['source_id']
                
                # reference_documents 테이블에서 참조 문서 조회
                reference_ids = get_reference_documents(cursor, source_id)
                
                # references JSON 배열 생성
                if reference_ids:
                    # [{"sourceId": "doc_111_02"}, {"sourceId": "doc_222_02"}, ...] 형태로 생성
                    references_json = json.dumps([
                        {"sourceId": f"{ref_id}"} 
                        for ref_id in reference_ids
                    ], ensure_ascii=False)
                    update_count += 1
                    print(f"[{idx}/{total_count}] ✓ {source_id}: {len(reference_ids)}개의 참조 문서 업데이트")
                else:
                    references_json = json.dumps([], ensure_ascii=False)
                    no_reference_count += 1
                    if idx % 100 == 0:  # 100개마다 한번씩만 출력
                        print(f"[{idx}/{total_count}] ○ 참조 문서 없음 (진행중...)")
                
                # documents 테이블의 references 컬럼 업데이트 (백틱 추가)
                update_query = """
                    UPDATE documents 
                    SET `references` = %s 
                    WHERE id = %s
                """
                cursor.execute(update_query, (references_json, doc_id))
            
            # 변경사항 커밋
            connection.commit()
            
            print(f"\n{'='*60}")
            print(f"작업 완료!")
            print(f"- 총 처리: {total_count}개 문서")
            print(f"- references 업데이트: {update_count}개")
            print(f"- 참조 문서 없음: {no_reference_count}개")
            print(f"{'='*60}")
            
    except Exception as e:
        print(f"✗ 오류 발생: {e}")
        connection.rollback()
    finally:
        connection.close()

# 실행
if __name__ == "__main__":
    update_documents_references()

총 23306개 문서 처리 시작...

[3/23306] ✓ 10002753: 1개의 참조 문서 업데이트
[18/23306] ✓ 10019691: 1개의 참조 문서 업데이트
[100/23306] ○ 참조 문서 없음 (진행중...)
[102/23306] ✓ 10121473: 1개의 참조 문서 업데이트
[200/23306] ○ 참조 문서 없음 (진행중...)
[300/23306] ○ 참조 문서 없음 (진행중...)
[380/23306] ✓ 10514006: 1개의 참조 문서 업데이트
[400/23306] ○ 참조 문서 없음 (진행중...)
[440/23306] ✓ 10630649: 1개의 참조 문서 업데이트
[500/23306] ○ 참조 문서 없음 (진행중...)
[523/23306] ✓ 10762617: 1개의 참조 문서 업데이트
[553/23306] ✓ 10806678: 1개의 참조 문서 업데이트
[568/23306] ✓ 10827785: 1개의 참조 문서 업데이트
[600/23306] ○ 참조 문서 없음 (진행중...)
[645/23306] ✓ 10921534: 1개의 참조 문서 업데이트
[658/23306] ✓ 10934514: 1개의 참조 문서 업데이트
[700/23306] ○ 참조 문서 없음 (진행중...)
[800/23306] ○ 참조 문서 없음 (진행중...)
[900/23306] ○ 참조 문서 없음 (진행중...)
[1000/23306] ○ 참조 문서 없음 (진행중...)
[1100/23306] ○ 참조 문서 없음 (진행중...)
[1200/23306] ○ 참조 문서 없음 (진행중...)
[1231/23306] ✓ 11859413: 1개의 참조 문서 업데이트
[1233/23306] ✓ 11861495: 1개의 참조 문서 업데이트
[1300/23306] ○ 참조 문서 없음 (진행중...)
[1400/23306] ○ 참조 문서 없음 (진행중...)
[1500/23306] ○ 참조 문서 없음 (진행중...)
[1600/23306] ○ 참조 문서 없음 (